In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate(); Pkg.add("Flux")

using Flux
using Flux: crossentropy, throttle
using Flux.Data: Tree, children, isleaf

include("data.jl")

N = 300

embedding = param(randn(N, length(alphabet)))

W = Dense(2N, N, tanh)
combine(a, b) = W([a; b])

sentiment = Chain(Dense(N, 5), softmax)

function forward(tree)
  if isleaf(tree)
    token, sent = tree.value
    phrase = embedding * token
    phrase, crossentropy(sentiment(phrase), sent)
  else
    _, sent = tree.value
    c1, l1 = forward(tree[1])
    c2, l2 = forward(tree[2])
    phrase = combine(c1, c2)
    phrase, l1 + l2 + crossentropy(sentiment(phrase), sent)
  end
end

loss(tree) = forward(tree)[2]

opt = ADAM(params(embedding, W, sentiment))
evalcb = () -> @show loss(train[1])

Flux.train!(loss, zip(train), opt,
            cb = throttle(evalcb, 10))

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/notebooks/text/treebank/Project.toml`
  [587475ba] ~ Flux v0.6.7+ #master (https://github.com/FluxML/Flux.jl.git) ⇒ v0.6.10+ #master (https://github.com/FluxML/Flux.jl.git)
  Updating `~/modelzoo/model-zoo/notebooks/text/treebank/Manifest.toml`
  [1520ce14] ↑ AbstractTrees v0.2.0 ⇒ v0.2.1
  [79e6a3ab] ↑ Adapt v0.3.1 ⇒ v0.4.1
  [b99e7846] ↑ BinaryProvider v0.4.2 ⇒ v0.5.3
  [944b1d66] ↑ CodecZlib v0.5.0 ⇒ v0.5.1
  [3da002f7] ↑ ColorTypes v0.7.4 ⇒ v0.7.5
  [5ae59095] ↑ Colors v0.9.4 ⇒ v0.9.5
  [34da2185] ↑ Compat v1.1.0 ⇒ v1.4.0
  [864edb3b] ↑ DataStructures v0.12.0 ⇒ v0.14.0
  [587475ba] ~ Flux v0.6.7+ #master (https://github.com/FluxML/Flux.jl.git) ⇒ v0.6.10+ #master (https://github.com/FluxML/Flux.jl.git)
  [f6369f11] ↑ ForwardDiff v0.9.0 ⇒ v0.10.1
  [e89f7d12] ↑ Media v0.4.1 ⇒ v0.5.0
  [e1d29d7a] ↑ Mis